# Data missing imputation considering a grouping

This class implements data imputation considering a grouping from a categorical variable. The data used in this example is related to the Australian weather. There are a lot of missing values and they will be filled by the mean, median and mode taking into consideration the location where the data were collected.

The data will be downloaded directly using the kaggle api, but you can find it [here](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) as well.

The code can be found [here](https://github.com/abreukuse/ml_utilities/blob/master/group_imputer.py) and it has fit and transform capabilities, so it can be used together with sklearn pipelines. Which means that the statistics will be calculated using only the training set, and then mapped to the validation/test set.

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline

from group_imputer import GroupImputer

import os
os.environ['KAGGLE_USERNAME'] = 'kaggle_username'
os.environ['KAGGLE_KEY'] = 'kaggle_api_key'

In [3]:
!kaggle datasets download -d jsphyg/weather-dataset-rattle-package

  0% 0.00/3.83M [00:00<?, ?B/s]
100% 3.83M/3.83M [00:00<00:00, 126MB/s]


In [4]:
!unzip 'weather-dataset-rattle-package.zip'

Archive:  weather-dataset-rattle-package.zip
  inflating: weatherAUS.csv          


In [5]:
data = pd.read_csv('weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [6]:
# Missing data in each variable
data.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [7]:
imputer_mean = GroupImputer(grouping='Location', 
                            
                            columns=['MaxTemp',
                                     'MinTemp',
                                     'Rainfall',
                                     'Evaporation',
                                     'Humidity9am',
                                     'Humidity3pm',
                                     'Temp9am',
                                     'Temp3pm'],
                            
                             strategy='mean')

imputer_median = GroupImputer(grouping='Location', 
                              
                              columns=['Sunshine',
                                       'WindSpeed9am',
                                       'WindSpeed3pm',
                                       'WindGustSpeed',
                                       'Pressure9am',
                                       'Pressure3pm'],
                              
                               strategy='median')

imputer_mode = GroupImputer(grouping='Location', 
                            
                            columns=['WindGustDir',
                                     'WindDir9am',
                                     'WindDir3pm',
                                     'Cloud9am',
                                     'Cloud3pm',
                                     'RainToday'],
                            
                             strategy='mode')

In [9]:
# There are missing data in the target
data = data.loc[~data['RainTomorrow'].isnull(), :].copy()

In [10]:
# Simple split
train = data.query('Date < "2015-01-01"').copy()
validation = data.query('Date >= "2015-01-01"').copy()

In [11]:
# Missing data in the training set
train.isnull().sum()

Date                 0
Location             0
MinTemp            434
MaxTemp            198
Rainfall          1000
Evaporation      37110
Sunshine         40696
WindGustDir       6943
WindGustSpeed     6902
WindDir9am        7323
WindDir3pm        2030
WindSpeed9am      1133
WindSpeed3pm      1140
Humidity9am       1265
Humidity3pm       1186
Pressure9am       9345
Pressure3pm       9309
Cloud9am         35764
Cloud3pm         36766
Temp9am            783
Temp3pm            663
RainToday         1000
RainTomorrow         0
dtype: int64

In [12]:
# Missing data in the validation set
validation.isnull().sum()

Date                 0
Location             0
MinTemp            203
MaxTemp            124
Rainfall           406
Evaporation      23733
Sunshine         27120
WindGustDir       2387
WindGustSpeed     2368
WindDir9am        2690
WindDir3pm        1748
WindSpeed9am       215
WindSpeed3pm      1490
Humidity9am        509
Humidity3pm       2424
Pressure9am       4669
Pressure3pm       4672
Cloud9am         17893
Cloud3pm         20328
Temp9am            121
Temp3pm           2063
RainToday          406
RainTomorrow         0
dtype: int64

In [13]:
# Data imputation procedire
pipeline = make_pipeline(imputer_mean, 
                         imputer_median, 
                         imputer_mode)

train_nan_imputed = pipeline.fit_transform(train)
validation_nan_imputed = pipeline.transform(validation)

In [14]:
train_nan_imputed.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,5.363867,8.35,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,8.0,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,5.363867,8.35,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,8.0,8.0,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,5.363867,8.35,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,5.363867,8.35,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,8.0,8.0,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,5.363867,8.35,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [16]:
validation_nan_imputed.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
2133,2015-01-01,Albury,11.4,33.5,0.0,5.363867,8.35,WSW,30.0,ESE,W,7.0,11.0,45.0,14.0,1013.5,1011.0,8.0,8.0,21.0,32.7,No,No
2134,2015-01-02,Albury,15.5,39.6,0.0,5.363867,8.35,NE,56.0,ESE,ESE,9.0,9.0,45.0,12.0,1016.0,1012.4,8.0,8.0,25.6,38.2,No,No
2135,2015-01-03,Albury,17.1,38.3,0.0,5.363867,8.35,NNE,48.0,NE,N,20.0,20.0,35.0,19.0,1017.9,1012.3,8.0,8.0,29.2,37.0,No,No
2136,2015-01-04,Albury,26.0,33.1,0.0,5.363867,8.35,NNE,41.0,ESE,W,7.0,7.0,46.0,37.0,1013.6,1012.1,8.0,5.0,27.4,30.9,No,No
2137,2015-01-05,Albury,19.0,35.2,0.0,5.363867,8.35,E,33.0,SSE,SE,7.0,9.0,60.0,34.0,1017.4,1014.7,8.0,8.0,25.6,32.5,No,No


In [17]:
train_nan_imputed.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [18]:
validation_nan_imputed.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64